In [1]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
import json
import os
from pprint import pprint
from sklearn.model_selection import train_test_split
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

# tokenize

In [22]:
data = pd.read_csv("TrainData.csv",sep=",",encoding = "utf-8")

In [24]:
train_data, test_data = train_test_split(data,test_size=0.2,random_state=42)

In [25]:
train_data = train_data.values
test_data = test_data.values

train_data = train_data.tolist()
test_data = test_data.tolist()

In [26]:
f = open('StopWords.txt','r',encoding = "utf-8")
stopwords = []
lines = f.readlines()
for line in lines:
    stopwords.append(line[:-1])
f.close()

In [27]:
okt = Okt()
def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return [t[0] for t in okt.pos(doc, norm=True, stem=True) if t[1] != 'Josa' and t[0] not in stopwords]

In [28]:
train = []
num = 0
for row in train_data:
    num += 1 
    try:
        t1 = tokenize(row[0])
        if row[1] > 5:
            sc = 1
        else:
            sc = 0
        train.append((t1,sc))
    except:
        print(num)
    
    if (num % 10000) == 0:
            print(num)


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000


In [29]:
with open('train_docs.json', 'w', encoding="utf-8-sig") as make_file:
    json.dump(train, make_file, ensure_ascii=False, indent="\t")

In [30]:
test = []
num = 0
for row in test_data:
    num += 1 
    try:
        t1 = tokenize(row[0])
        if row[1] > 5:
            sc = 1
        else:
            sc = 0
        test.append((t1,sc))
    except:
        print(num)
    
    if (num % 10000) == 0:
            print(num)


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000


In [31]:
 with open('test_docs.json', 'w', encoding="utf-8-sig") as make_file:
        json.dump(test, make_file, ensure_ascii=False, indent="\t")

# term frequency

In [15]:
if os.path.isfile('train_docs.json'):
    with open('train_docs.json',encoding='utf-8-sig') as f:
        train_docs = json.load(f)
    with open('test_docs.json',encoding='utf-8-sig') as f:
        test_docs = json.load(f)
else:
    train_docs = []
    test_docs = []
    for row in train_data:
        try:
            t = tokenize(row[0])
            train_docs.append((t,row[1]))
        except:
            print(row[0])
    for row in test_data:
        try:
            t = tokenize(row[0])
            test_docs.append((t,row[1]))
        except:
            print(row[0])
    # JSON 파일로 저장
    with open('train_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(train_docs, make_file, ensure_ascii=False, indent="\t")
    with open('test_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(test_docs, make_file, ensure_ascii=False, indent="\t")

In [16]:
tokens = [t for d in train_docs for t in d[0]]
print(len(tokens))

4608106


In [17]:
import nltk
text = nltk.Text(tokens, name='NMSC')

selected_words = [f[0] for f in text.vocab().most_common(10000)]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

In [24]:
train_x = [term_frequency(d) for d, _ in train_docs1]

In [18]:
test_x = [term_frequency(d) for d, _ in test_docs]
train_y = [c for _, c in train_docs]
test_y = [c for _, c in test_docs]

# deep learning

In [ ]:
train_x = np.asarray(train_x)

In [ ]:
test_x = np.asarray(test_x)

train_y = np.asarray(train_y)
test_y = np.asarray(test_y)

In [ ]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

model.fit(train_x, train_y, epochs=10, batch_size=512)
results = model.evaluate(test_x, test_y)

In [ ]:
results

In [ ]:
def predict_pos_neg(review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    score = float(model.predict(data))
    if(score > 0.5):
        print("[{}]는 {:.2f}% 확률로 긍정 리뷰\n".format(review, score * 100))
    else:
        print("[{}]는 {:.2f}% 확률로 부정 리뷰\n".format(review, (1 - score) * 100))